# RAG Project

LangChain Tutorials: https://python.langchain.com/docs/tutorials/

LangChain HowTos: https://python.langchain.com/docs/how_to/

LangChain Conceptual Guide: https://python.langchain.com/docs/concepts/#retrieval

High-Level Overview of RAG: https://python.langchain.com/docs/tutorials/rag/



In [1]:
%pip install --quiet --upgrade bitsandbytes langchain langchain-community langchain-huggingface transformers beautifulsoup4 faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 100.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.

## **Part 1: Setup LLM**

**Set up LLM**
- https://python.langchain.com/docs/integrations/providers/huggingface/

<br/>
<br/>

<u>**Experimentation and Improvements** (not very important)</u>
- Find a better LLM that also fits into memory
  - Mistral-Small-Instruct-2409: Positioned Between Mistral NeMo 12B and Mistral Large 123B (crashed)
    - https://www.marktechpost.com/2024/09/18/mistral-ai-released-mistral-small-instruct-2409-a-game-changing-open-source-language-model-empowering-versatile-ai-applications-with-unmatched-efficiency-and-accessibility/
    - https://huggingface.co/mistralai/Mistral-Small-Instruct-2409
  - Mistral Nemo (Mistral-Nemo-Instruct-2407) (12 mins with quantization)
      - https://www.reddit.com/r/LocalLLaMA/comments/1eg5j2t/which_small_model_12b_do_you_guys_are_using_for/
      - https://huggingface.co/mistralai/Mistral-Nemo-Instruct-2407
  - Mistral 7B Quantized, Mistral 7B Instruct
    - https://www.reddit.com/r/LocalLLaMA/comments/1av779p/experiences_with_smaller_models_with_rag/

  - Qwen2–72B-Instruct, Qwen1.5-32B-Chat
      - https://medium.com/@naman1011/whats-the-best-llm-to-use-for-rag-476bec1bfa97
      - https://huggingface.co/Qwen/Qwen2.5-72B-Instruct

Quantization
- https://medium.com/@rakeshrajpurohit/model-quantization-with-hugging-face-transformers-and-bitsandbytes-integration-b4c9983e8996

In [2]:
import torch
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import BitsAndBytesConfig

#from huggingface_hub import notebook_login
#notebook_login()

# We load the quantized weights for faster generation
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

model_id = "mistralai/Mistral-Nemo-Instruct-2407"
# model_id = "Qwen/Qwen2.5-0.5B-Instruct" # choose this for faster inference
tokenizer = AutoTokenizer.from_pretrained(model_id)
#model = AutoModelForCausalLM.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,quantization_config=quantization_config)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer,max_new_tokens=1000)
llm = HuggingFacePipeline(pipeline=pipe)

tokenizer_config.json:   0%|          | 0.00/181k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.26M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


...

model-00003-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

## **Part 2: Load Document**

Load Data
- https://python.langchain.com/docs/how_to/#document-loaders

Can try loading data from web pages using the <em>Simple and Fast Parsing Approach</em> to recover one ```Document``` object per webpage
- https://python.langchain.com/docs/how_to/document_loader_web/
- https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.web_base.WebBaseLoader.html
- https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html

Going to use text document because the web one had too much irrelevant info
- https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.text.TextLoader.html

<br/>
<br/>

<u>Concern for Data collection</u>

- Example: When we ask the model what are the best places for hiking, and we have 10+ documents for hiking destinations, will that be a problem? Maybe we need metadata to denote the location
- Example: When we have 1 document that is all about hiking like 20 best hikes, how much is to be retrieved? -> Experiment with chunk and overlap size, parent-document retrieval approaches

<br/>
<br/>

<u>**Experimentation and Improvements**</u>

<u>1. Decide the categories of documents that we are going to put in (by country) and how many of each</u>
- Activities
- Food
- Accomodation
- Transport
- Places of interest
- etc.

<u>2. Scrape the web better</u>
- Use advanced parsing
- Use lazy loading and async for efficiency
- Specify parameters for BeautifulSoup through bs_kwargs to pick up only body text instead of extra info like navigation bars i.e. Parse web pages better

<u>3. Instead of scraping web, use other sources such as text documents</u>
- Find another way to get data

<u>4. Organize data (either local folder of text documents or through cloud or DB?)</u>

In [4]:
"""
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_path = "https://www.earthtrekkers.com/norway-bucket-list-best-things-to-do-in-norway/"
)

docs = loader.load()
"""

from langchain_community.document_loaders import TextLoader
# Change to reading by a directory with all our text files if we are really going ahead with reading from text files
loader = TextLoader(
    file_path = "Norway Example.txt"
)

docs = loader.load()

In [5]:
docs

[Document(metadata={'source': 'Norway Example.txt'}, page_content='15 of the Most Beautiful Fjord Hikes in Norway\n\nGlacial lagoons, verdant forests and soaring mountains - our guide to the best fjord hikes in Norway...\n\nLooking for the best fjord hikes in Norway? Hiking is second nature to Norwegians and with the vast and varied landscapes on their doorstep, it’s easy to see why. From the white-sand beaches of the Lofoten Islands to the dramatic mountains of the south, and the spectacular fjords in between, there’s something to suit all types of hiker. It\'s Norway\'s fjords we\'re going to focusing on today - because many of the best hiking in Norway happens to be along, around or above a fjord.\n\nMany fjords promise spectacular natural views of untouched verdant forests, ice cold lagoons and sweeping views over seemingly endless cliffs and mountains. They\'re also great places to chase the Northern Lights during winter. Here, we\'re going to give you an introduction to the best 

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

## **Part 3: Chunking/Split Text**

Split LangChain ```Document``` objects into smaller chunks. This is useful for
- Indexing and retrieving relevant data
- LLM (cannot fit into model's finite context window)
- Overcoming embedding model size limitations
- https://python.langchain.com/docs/concepts/text_splitters/
- https://python.langchain.com/docs/how_to/#text-splitters

Try ```RecursiveCharacterTextSplitter``` first
- Overlap helps to mitigate the possibility of separating a statement from important context related to it
- ```RecursiveCharacterTextSplitter``` recursively split the document using common separators like new lines until each chunk is the appropriate size. This is the recommended text splitter for generic text use cases
- Set ```add_start_index=True``` so that the character index at which each split Document starts within the initial Document is preserved as metadata attribute ```“start_index”```.
- This is a text-structured based splitting approach: Creating split that maintain natural language flow, maintain semantic coherence within split, and adapts to varying levels of text granularity

<br/>
<br/>

<u>**Experimentation and Improvements**</u>
1. Clean Data
2. Add metadata to chunks
  - This is for self query: https://python.langchain.com/docs/how_to/self_query/
  - I think some useful metadata that could be useful could be activity type and country. Not sure about other metadata
3. Experiment with Chunk size and Overlap size for recursive text splitting
4. Experiment with other splitting strategies: Document-structured based splitting, Semantic meaning based splitting (not sure if relevant)

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True # Following tutorial specs
)
all_splits = text_splitter.split_documents(docs)

In [7]:
all_splits

[Document(metadata={'source': 'Norway Example.txt', 'start_index': 0}, page_content="15 of the Most Beautiful Fjord Hikes in Norway\n\nGlacial lagoons, verdant forests and soaring mountains - our guide to the best fjord hikes in Norway...\n\nLooking for the best fjord hikes in Norway? Hiking is second nature to Norwegians and with the vast and varied landscapes on their doorstep, it’s easy to see why. From the white-sand beaches of the Lofoten Islands to the dramatic mountains of the south, and the spectacular fjords in between, there’s something to suit all types of hiker. It's Norway's fjords we're going to focusing on today - because many of the best hiking in Norway happens to be along, around or above a fjord.\n\nMany fjords promise spectacular natural views of untouched verdant forests, ice cold lagoons and sweeping views over seemingly endless cliffs and mountains. They're also great places to chase the Northern Lights during winter. Here, we're going to give you an introduction

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

## **Part 4: Indexing/Storing chunks/splits**

Embed the contents of each chunk/split and insert these embeddings into a vector store
- When we want to search over our splits, we take a text search query, embed it, and perform some sort of “similarity” search to identify the stored splits with the most similar embeddings to our query embedding.
- Simplest similarity measure is cosine similarity. We measure the cosine of the angle between each pair of embeddings (which are high dimensional vectors).
- https://python.langchain.com/docs/how_to/
- https://python.langchain.com/docs/how_to/embed_text/
- https://python.langchain.com/docs/concepts/embedding_models/
- https://python.langchain.com/docs/how_to/vectorstores/
- https://python.langchain.com/docs/concepts/vectorstores/
- https://python.langchain.com/docs/integrations/vectorstores/
  - https://python.langchain.com/docs/integrations/vectorstores/faiss/
  - https://python.langchain.com/api_reference/community/vectorstores/langchain_community.vectorstores.faiss.FAISS.html

Embedding vectors (capture semantic meanings) can be compared easily
- Use ```embed_documents``` to embed multiple texts (documents)
- Use ```embed_query``` to embed a single text (query)
- As they are a set of coords in high-dim space, their similarity can be measured by
  - Cosine Similarity, Euclidean Distance, Dot Product

Vector Store
- Takes care of storing embedding vectors and performing similarity vector search against embedded query, therefore helping to retrieve relevant information based on semantic similarity
- Standard interface for working with vector stores: ```add_documents```, ```delete_documents```, ```similarity_search```
- Choice of similarity metric can sometimes be selected when initlaising vector store: refer to documentation of specific vectorstore we are using
- Choice of similarity search: Given the similarity metric to measure distance between the embedded query and embedded chunks, we need an algo to efficiently search over all embedded chunks. Many vector stores implement ```Hierarchical Navigable Small World```, a graph-based index structure that allows for eficient similarity search. See what search parameters we can add for specific choice of vector store
- Many vector stores support metadata filtering: Allows structured filters to reduce size of similarity search space
- Semantic search and Metadata search work hand in hand

<br/>

For now, use Hugging Face's ```sentence-transformers/all-mpnet-base-v2``` embedding model and ```FAISS``` as the vector store

<br/>
<br/>

<u>**Experimentation and Improvements**</u>

1. Explore other indexes used by FAISS
  - In our HW we played with Nbits for LSH
  - Compared cosine similarity, search time, precision, recall when changing Nbits
2. Explore Hierarchical Navigable Small World (HNSW), knowledge graphs
3. Explore other vector DBs like Chroma, Spotify's Annoy (not a priority unless their search methods are novel)
4. Explore other embedding models (not a priority)
5. IR System Evaluation Metrics (Must do)
  - Binary Relevance
  - Multiple Levels of Relevance

Note: Comparison of FAISS with chroma
- https://medium.com/@stepkurniawan/comparing-faiss-with-chroma-vector-stores-0953e1e619eb




In [8]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore

embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

index = faiss.IndexFlatL2(len(embeddings_model.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings_model,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

vector_store.add_documents(all_splits)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

['d49596f6-8f31-4f21-b055-f38fba0cbbd4',
 '68d44cf3-f790-4ce4-9da4-f7d420e15da7',
 'c22be216-987d-4b46-b1a0-224ef063156b',
 'b9f94fad-72f0-4bcb-807b-364f404deaac',
 'c3f037c7-0a10-4fae-99ab-5a2c96880d5a',
 'c296da7b-157e-44aa-b29d-0f166ea04ab2',
 '6bf9b5f7-e568-41eb-af4a-951332232cde',
 '3eab7335-16c9-4be6-a00e-11a8932a026c',
 '8e26dbec-e58c-4c66-a4c6-b38ec228b2e3',
 'f9dccba1-08a5-4ae5-b75d-7421e16b1b7b',
 '52f818b5-6b49-41c6-b737-77a2031c3231',
 'dee565f9-c45c-4e7b-9b81-d75332dcc55c',
 'b1eabe8f-5319-4ae9-8411-ef41c20bef7e',
 'f73fa1f5-51ea-4d77-84c1-31c8fc815226',
 '6a269c5c-1bb7-4736-b0ce-9399cf24b938',
 'c7099369-22b7-4996-89ab-48e66bafb3b9',
 '09df9862-3468-4593-abc5-58ff7415d799',
 'fd5f31b7-2c34-4ace-8362-83060b44a313',
 'bb409ba4-f6c3-4517-8069-ba6c0ed48cbc',
 '91fbcdb2-7a55-44b2-94f0-a69bb8da3e4c',
 'b7fa1077-c404-4b75-b57d-d5f7cc8d891d',
 'e3fbae0c-d614-40cf-9f2f-d502b081a2ba',
 'f0fc0910-fe27-48e9-a6db-9043a86ad154',
 '85befd53-76af-417a-a8ca-c09715a23b78',
 'c4dd5de8-77ae-

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

## **Part 5: Retrieval**

Given a user query, retrieve relevant splits from our vector store
- Experimentations (See experimentation and improvements section): https://python.langchain.com/docs/how_to/#retrievers

```Retriever``` interface wraps an index
- Uniform interface for interacting with different types of retrieval systems: Vector stores, Graph databases, Relational databases
- Input: Query string
- Output: List of LangChain Document objects
- Its a runnable (standard interface for LangChain components). Therefore, it has the method ```invoke()``` to invoke it with a query
- A vectorstore can be used as a retriever by calling the ```as_retriever()``` method
- https://python.langchain.com/docs/concepts/retrievers/

Query Analysis: Models transform or construct search queries from raw user input to optimise retrieval
- https://python.langchain.com/docs/concepts/retrieval/
- Query Re-writing
  - Multi-query
  - Decomposition
  - Step-back
  - HyDE
- Query Construction
  - Self Query (use metadata)

Information Retrieval: Search queries are used to fetch information from various retrieval systems.
- https://python.langchain.com/docs/concepts/retrieval/
- Lexical search indexes (based on keywords). Data structure to implement this is called inverted index. Lexical search algorithms include BM25
- Vector indexes (based on word embeddings)


Most common type of ```Retriever``` is the ```VectorStoreRetriever```: Uses the similarity search capabilities of a vector store to facilitate retrieval. Any VectorStore can easily be turned into a Retriever with ```VectorStore.as_retriever()```

<br/>
<br/>


<u>**Experimentation and Improvements**</u>
1. Ensemble/Hybrid Search: Combine multiple retrievers. Particularly useful when you have multiple retrievers that are good at finding different types of relevant documents. We could fist try combining a sparse retriever (BM25 that is keyword based) and dense retriever (embedding similarity)
  - https://python.langchain.com/docs/concepts/retrievers/
  - https://python.langchain.com/docs/how_to/ensemble_retriever/
  - https://python.langchain.com/docs/how_to/hybrid/

2. Source Document Retention Approaches: Retain a link between the transformed document (chunks) and original document, giving the retriever the ability to return the original document. For example, you may use small chunk size for indexing documents in a vectorstore. If you return only the chunks as the retrieval result, then the model will have lost the original document context for the chunks.
  - Multi-Vector retriever: https://python.langchain.com/docs/how_to/multi_vector/
  - ParentDocument retriever: https://python.langchain.com/docs/how_to/parent_document_retriever/

3. Query Rewriting Approaches:
  - https://python.langchain.com/docs/concepts/retrieval/
  - Multi-query
  - Decomposition
  - Step-back
  - HyDE

4. Query Construction Approaches:
  - https://python.langchain.com/docs/concepts/retrieval/
  - Self query

5. Others:
  - Contextual Compression
  - Reorder/Rank Documents
    - https://python.langchain.com/docs/how_to/long_context_reorder/
    - RSV, RSV with smoothing (assume relevance feedback)
    - IDF (assume relevance feedback)
    - BM25 (assume no relevance feedback)
  - Maximal marginal relevance?
  - Multi Vector Retrieval?: https://python.langchain.com/docs/how_to/multi_vector/

6. IR System Evaluation Metrics (Must do)
  - Binary Relevance
  - Multiple Levels of Relevance


A list of techniques in LangChain:
- https://python.langchain.com/docs/how_to/#retrievers


In [9]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 10})

retrieved_docs = retriever.invoke("What are the best hikes in Norway?")

In [10]:
retrieved_docs[0].page_content

"In many countries, the best hiking routes are only obtainable to those in the know or with high-tech equipment – this is not the case in Norway, the undisputed home of the fjords! However, with many of the best Norway fjord hikes accessible only by boat or kayak, we would recommend travelling with a guide where possible.\n\nHere are 15 fjords worth hiking on your next Norway walking adventure. These are all day hikes with breathtaking views, many of which are accessible by public transport.\n\n1. Preikestolen, Lysefjord\n\n2. Breiskrednosi Summit Hike, Naerøyfjord\n\n3. Romsdalseggen Ridge, Isfjorden\n\n4. Mount Skåla, Nordfjord\n\n5. Trolltunga (aka The Devil's Tongue), Hardangerfjord\n\n6. Aurlandsdalen Valley, Sognefjord\n\n7. Dronningstien, Hardangerfjord\n\n8. Vidasethovden, Sognefjord\n\n9. Mount Hanguren, Sognefjord/ Hardangerfjord\n\n10. Mount Fløyen, Sognefjord/ Hardangerfjord\n\n11. Mount Ulriken, Sognefjord/ Hardangerfjord\n\n12. Rimstigen, Næroyfjord\n\n13. Urkeegga, Hjøru

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

## **Part 6: Generate**

<u>Overview of Generate Step</u>

An LLM produces an answer using a prompt that includes the question and retrieved data

A chain in LangChain is a sequence of interconnected components that process a user's query to generate and deliver valuable output. It is like a pipeline

The components of a chain typically include:

- Prompts: Templates that guide the LLM's responses.

- LLMs or Chat Models: The engines that generate responses based on the prompts.

- Output Parsers: Tools that parse the LLM's output.

- Tools: Extensions that allow LLMs to extract additional information from APIs or run code, turning LLMs into agents.

- General Functions: Additional functionalities that can be chained together.

In LangChain, these components, known as runnables, can be combined/chained to form a comprehensive pipeline. Pipelines are a way to think of chains

An LLM chain is similar to a data pipeline. In a data pipeline, raw data is transformed into clean, structured data.

Similarly, in an LLM chain, a query is transformed into valuable output using LLM calls, functions, and additional data.

<br/>

<u>More on Prompts</u>

A prompt for a language model is a set of instructions provided by a user to guide the model's response, helping it to understand the context and generate relevant and coherent output

Prompt templates in LangChain are predefined receipes for generating prompts for language models

```
from langchain.prompts import PromptTemplate

# Create a simple prompt template
# Formulate the prompt to the LLM with a string as a placeholder, in this case {topic}
prompt_template = """
You are a helpful assistant that explains AI topics. Given the following input:
{topic}
Provide an explanation of the given topic.
"""

# Create the prompt from the prompt template from LangChain
prompt = PromptTemplate(
    input_variables=["topic"],
    template=prompt_template,
)

# Assemble the chain using the pipe operator "|", more on that later. Assemble a chain using both components
chain = prompt | llm

# Invoke the chain with an input variable
chain.invoke({"topic":"What is LangChain"})
```

LangChain rag-prompts from prompt hub:
https://smith.langchain.com/hub/rlm/rag-prompt

<br/>

<u>LangChain Expression Language (LCEL)</u>

LangChain Expression Language (LCEL) simplifies building complex chains from basic components.

It uses the pipe operator (|) to chain different components, feeding the output from one component as input to the next.

 A simple example of a chain composed this way would be a prompt combined with a model and an output parser.

 ```chain = prompt | model | output_parser```

 These components are called "runnables". Think of LangChain Expression Language(LCEL) as a declarative way of composing these runnables into chains.

 We can use the string output of this chain and send it to a new chain that will return a different output

LangChain allows multiple chains to be chained together in this manner.

```chain = chain_1 | chain_2```

The Runnable Protocol

A runnable is a unit of work that can be invoked, batched, streamed, transformed, and composed.

The chains we build with LangChain and their components (the components of those chains) are runnables.

We can also pass arbitrary functions into a chain, which will be converted into runnables.

```chain = prompt | (lambda input: {"x":input}) | model | output_parser```

These chains can be:

```
chain.invoke(...)
chain.batch([...]) if we have multiple inputs
chain.stream(...)
```

Each of these components (```retriever, prompt, llm, etc.```) are instances of Runnable. This means that they implement the same methods - such as sync and async ```.invoke```, ```.stream```, or ```.batch``` - which makes them easier to connect together. They can be connected into a ```RunnableSequence```--another Runnable--, via the ```|``` operator.

LangChain will automatically cast certain objects to runnables when met with the | operator.

Core runnable objects in LangChain:

RunnableSequence: A class that chains together multiple runnable components, ensuring each component processes its input and sequentially passes its output to the next component in the pipeline.

RunnableLambda: A class that turns a Python callable (like a function) into a runnable component, allowing integration of arbitrary functions into chains.

RunnablePassthrough: A class that either passes its input through unchanged or adds additional keys to the output. It can act as a placeholder or allow flexible integrations into sequences where we need to modify the input.

RunnableParallel: A class that runs multiple runnables concurrently, allowing branching where two chains run on the same input but return different outputs (return a mapping of their outputs).

<br/>

<u>Experimentation and Improvements</u>
1. Citations (Must do)
2. Prompt Engineering?



In [11]:
from langchain import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Question: {question}
Context: {context}
Helpful Answer:
"""

prompt = PromptTemplate.from_template(template)

In [12]:
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="\nYou are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nQuestion: {question}\nContext: {context}\nHelpful Answer:\n")

In [13]:
output_parser = StrOutputParser()

chain = (
    {"context": (lambda x: x["question"]) | retriever,

     "question": (lambda x: x["question"])}
    | prompt
    | llm
    | StrOutputParser()
)

In [14]:
answer = chain.invoke({"question":"What are the best things to do in Norway?"})

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [15]:
print(answer)


You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Question: What are the best things to do in Norway?
Context: [Document(metadata={'source': 'Norway Example.txt', 'start_index': 1786}, page_content="9. Mount Hanguren, Sognefjord/ Hardangerfjord\n\n10. Mount Fløyen, Sognefjord/ Hardangerfjord\n\n11. Mount Ulriken, Sognefjord/ Hardangerfjord\n\n12. Rimstigen, Næroyfjord\n\n13. Urkeegga, Hjørundfjord\n\n14. Himakånå, Nedstrandsfjorden\n\n15. Langfoss Waterfall Hike, Åkrafjord\n\nWe've suggested some day hikes within the fjords, but given that wild camping is permissible in Norway we also recommend embarking on a multi-day adventure.\n\nPlease note that the route descriptions we have included are summaries, designed to inspire rather than for any navigational purpose. Further mapping and planning - or an experienced local guide - will be needed to safely walk 

The best things to do in Norway include hiking in the fjords. Some of the best fjords for hiking are Nærøyfjord, Hjørundfjord, Sognefjord, and Hardangerfjord. These fjords offer excellent summit hikes with stunning views. Some popular hikes include Preikestolen, Breiskrednosi Summit Hike, Romsdalseggen Ridge, Mount Skåla, Trolltunga, Aurlandsdalen Valley, Dronningstien, Vidasethovden, Mount Hanguren, Mount Fløyen, Mount Ulriken, Rimstigen, Urkeegga, Himakånå, and Langfoss Waterfall Hike. These hikes offer a range of difficulties and are accessible by public transport. The best time to visit Norway's fjords for hiking is from spring to October, with peak season being mid-June to the end of July. However, weather can be changeable, so it's important to pack accordingly.

In [16]:
chain_skipprompt = (
    {"context": (lambda x: x["question"]) | retriever,

     "question": (lambda x: x["question"])}
    | prompt
    | llm.bind(skip_prompt=True)
    | StrOutputParser()
)

In [17]:
answer = chain_skipprompt.invoke({"question":"What are the best things to do in Norway?"})

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [18]:
print(answer) # Check why is this repeated for some LLMs?

The best things to do in Norway include hiking in the fjords. Some of the best fjords for hiking are Nærøyfjord, Hjørundfjord, Sognefjord, and Hardangerfjord. These fjords offer excellent summit hikes with stunning views. Some popular hikes include Preikestolen, Breiskrednosi Summit Hike, Romsdalseggen Ridge, Mount Skåla, Trolltunga, Aurlandsdalen Valley, Dronningstien, Vidasethovden, Mount Hanguren, Mount Fløyen, Mount Ulriken, Rimstigen, Urkeegga, Himakånå, and Langfoss Waterfall Hike. These hikes offer a range of difficulties and are accessible by public transport. The best time to visit Norway's fjords for hiking is from spring to October, with peak season being mid-June to the end of July. However, weather can be changeable, so it's important to pack accordingly.


Comparing it to no-RAG LLM response

In [19]:
print(llm.invoke("What are the best things to do in Norway??"))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


What are the best things to do in Norway???

Norway is a country known for its stunning natural beauty, rich history, and vibrant culture. Here are some of the best things to do in Norway:

1. **Visit the Fjords**: Norway's fjords are one of its most iconic natural features. The most famous is the Geirangerfjord, but others like the Hardangerfjord and the Sognefjord are also breathtaking. You can explore them by boat, kayak, or even hike along their edges.

2. **Explore Bergen**: This historic city is a gateway to the fjords and offers a mix of cultural attractions, museums, and vibrant neighborhoods. Don't miss the Bryggen Hanseatic Wharf, a UNESCO World Heritage site.

3. **Experience the Northern Lights (Aurora Borealis)**: If you're lucky enough to visit Norway between September and March, you might get to see this natural light show. The best places to view them are in the north, such as Tromsø or the Lofoten Islands.

4. **Visit Oslo**: Norway's capital offers a blend of history,